# A FEM implementation of "On a Wildland Fire Model with Radiation" using Firedrake

Asensio, M.I. and Ferragut, L. (2002), On a wildland fire model with radiation. Int. J. Numer. Meth. Engng., 54: 137-157. https://doi.org/10.1002/nme.420

In [1]:
from firedrake import *
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants

In [2]:
# Mesh parameters
nx = 50
ny = 50
Lx = 10
Ly = 10
mesh = RectangleMesh(nx, ny, Lx, Ly)
x, y = SpatialCoordinate(mesh)

In [3]:
# Simulation time parameters
time = 5.
num_steps = 500
dt_step = time / num_steps
dt = Constant(dt_step)

In [4]:
# Define the wind field
w_uv = as_vector([5, 0])
W = VectorFunctionSpace(mesh, family='CG', degree=2, dim=2)
w = interpolate(w_uv, W)

In [5]:
# Model Constants
T_pc = Constant(550)  # Remperature Phase change, K
T_inf = Constant(295.372)  # Ambient Temperature, K
rho = Constant(1.1774)  # Atmospheric Density, kg/m^3
C = Constant(1.0057)  # Atmospheric Specific Heat (kJ/kG)K
sigma = Constant(constants.sigma)  # 
dx = Constant(Lx / nx)
H = Constant(15900) # Heat of Combustion of Cellulose, kcal/kg
A = Constant(10 ** 9) # pre-exponential factor, 1/s 
E_A = Constant(83.68) # Activation Energy, kJ/mol
R = Constant(constants.R)
h = Constant(0.1) # Who knows?

In [6]:
# Model Functions
def s(T):
    return conditional(ge(T, T_pc), 1, 0)

In [7]:
# Define the mixed elements
P1 = FiniteElement('P', triangle, 1)
element = MixedElement([P1, P1])

# Define the test functions
V = FunctionSpace(mesh, element)
v_1, v_2 = TestFunctions(V)

# No trial functions since the problem is nonlinear
u = Function(V)
u_n = Function(V)
T, Y = split(u)
T_n, Y_n = split(u_n)

In [23]:
# Initial Conditions
r_center = sqrt((x - Lx / 2) ** 2 + (y - Ly / 2) ** 2)  # radial distance from center of domain
u_max = 10  # Stability is strongly related to this value, make dt smaller if this higher
radius = 4.0
smooth = 1.5
uic = u_max * (1 - 1 / (1 + exp(-smooth * (r_center - radius))))
u.sub(0).assign(project(uic, V[0]))
u.sub(1).assign(project(Constant(1), V[1]))

Coefficient(WithGeometry(IndexedProxyFunctionSpace(<firedrake.mesh.MeshTopology object at 0x10956ac10>, FiniteElement('Lagrange', triangle, 1), name=None, index=1, component=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 1)), 29)

In [24]:
# Variational Form
convection = (rho * C * (((T - T_n)/dt) + dot(w, grad(T))))* v_1 * dx
radiation = dot((Constant(4) * sigma * dx * pow(T, 3)) * grad(T), grad(v_1)) * dx
reaction = s(T) * H * A * exp(-E_A/(R * T)) * rho * Y * v_1 * dx
natural_convection = h * (T - T_inf) * v_1 * dx

mass_frac = ((Y - Y_n)/dt) * v_2 * dx
second_reaction = s(T) * Y * A * exp(-E_A/(R * T)) * v_2 * dx

F = convection - radiation - reaction + natural_convection + mass_frac + second_reaction

In [29]:
t = 0
while t <= time:
    solve(F == 0, u)
    u.assign(u_n)
    t += dt


TypeError: _la_solve() missing 1 required positional argument: 'b'